In [3]:
# import preprocess as prep
import SLR_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# from preprocess import person

# model will output multiple(15) results/sec, how are we gonna handle it?

In [4]:
importlib.reload(SLR_model)
importlib.reload(preprocess)


<module 'preprocess' from 'C:\\Users\\shd\\Documents\\capstone\\BomNae-SLR\\preprocess.py'>

In [5]:
# Configurations
model_dir = "saves"
THRESHOLD = 0.5
# batch_size = 16

In [7]:
# 세이브된 모델 로드
load_path = os.path.join(model_dir, "check_13-15-10.keras")
model = SLR_model.load_model(load_path)

OSError: Unable to open file (file signature not found)

In [16]:
# load data
l_raws=[]
r_raws=[]
p_raws=[]
y_raws=[]
person_id = 1
# TODO hardcode path
# a video
for j in os.listdir(os.path.join(preprocess.getoutputdir(),str(person_id))):
    # l_raw, r_raw, p_raw, y_raw = preprocess.load_data(f"{person_id}/{j}","val")
    l_raw, r_raw, p_raw, y_raw ,w= preprocess.load_data(f"{person_id}/0001.npz")
    
    l_raws.append(l_raw)
    r_raws.append(r_raw)
    p_raws.append(p_raw)
    y_raws.append(y_raw)
    break

In [17]:
y_raws

[1]

In [7]:
# # dummy x_raw data
# l_raws = []
# duration_list = []
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = (np.random.randint(90,100))
# duration_list.append(duration)
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# l_raws.append(x_elem)
# r_raws = []
# 
# h_elem = np.zeros(shape=(4,5,3))
# # p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     x_elem.append(h_elem)
#     # x_elem.append(comb_elem)
# r_raws.append(x_elem)
# p_raws = []
# 
# # h_elem = np.zeros(shape=(4,5,3))
# p_elem = np.zeros(shape=(3,10))
# # comb_elem = [h_elem, h_elem, p_elem]
# x_elem = []
# duration = duration_list[0]
# for j in range(duration): # frame size
#     # x_elem.append(h_elem)
#     x_elem.append(p_elem)
#     # x_elem.append(comb_elem)
# p_raws.append(x_elem)

In [18]:
l_test, each = SLR_model.serialize(l_raws)
r_test, each = SLR_model.serialize(r_raws)
p_test, each = SLR_model.serialize(p_raws, stride=2)
x_test = (l_test, r_test, p_test)

# dataset = SLR_model.convert_to_dataset_single(x_test, batch_size)


# with open(hist_path, 'w') as file:
#     json.dump(hist.history, file)

In [19]:
x_test[1].shape

(13, 63, 4, 5, 3)

In [26]:
predictions = model.predict(x_test,20000)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [27]:
predictions

array([[0.9972665 , 0.00550463, 0.9811525 , 0.04739995, 0.58294106,
        0.7283279 , 0.28779075, 0.34996867, 0.43131068, 0.4920706 ,
        0.39550504, 0.5136719 ],
       [0.9975734 , 0.00430226, 0.98124295, 0.06111937, 0.5441983 ,
        0.71501744, 0.33799925, 0.38855624, 0.501485  , 0.40314984,
        0.34051904, 0.53294003],
       [0.9974104 , 0.00556213, 0.98228467, 0.05866056, 0.60061973,
        0.68422997, 0.38042587, 0.4402206 , 0.5469699 , 0.39313254,
        0.37373698, 0.63316894],
       [0.9949049 , 0.01855318, 0.97066814, 0.13571696, 0.65392745,
        0.5969317 , 0.3356792 , 0.5119786 , 0.62832034, 0.28953528,
        0.5212295 , 0.6527545 ],
       [0.9840177 , 0.02855054, 0.96343833, 0.18937978, 0.72069556,
        0.51407456, 0.32745072, 0.55160505, 0.67583835, 0.30795038,
        0.49805522, 0.6416124 ],
       [0.9471832 , 0.03553902, 0.9548898 , 0.26721504, 0.7671008 ,
        0.41927218, 0.39092115, 0.5949038 , 0.7448019 , 0.41515732,
        0.385153  ,

In [28]:
pred_bin = predictions.round().astype(np.int32)
pred_bin

array([[1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]])

In [29]:
importlib.reload(SLR_model)

<module 'SLR_model' from 'C:\\Users\\shd\\Documents\\capstone\\BomNae-SLR\\SLR_model.py'>

In [30]:
SLR_model.bin2d_to_num(pred_bin.astype(np.int32))

[2753, 2761, 2761, 2779, 2777, 2713, 2717, 2717, 2717, 2717, 2749, 2749, 2717]

In [17]:
SLR_model.bin2num([1,0,1,1,1,0,1,1,1,0,1,0])

3002

In [ ]:
model.predict